In [1]:
import numpy as np
import tensorflow as tf
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.utils import shuffle
from scipy import stats
from scipy import optimize
import joblib
import matplotlib.pyplot as plt

#directory for saving results
filepath = ".../Resultate_final/Put/IS/test/Put_VaR_ES_IS_1_test_saved/"

In [2]:
#Market and option parameters as in section 4.1 of 'Assessing Asset-Liability Risk with Neural Networks' (Cheridito, Ery, Wüthrich 2020)
s_0 = 100
r = 0.01
mu = 0.05
sigma = 0.2
tau = 1/52
T = 1/3
K = 100

#quantile for which the IS distribution is computed
alpha_IS = 0.995
#Confidence levels for Value-at-Risk and Expected Shortfall
alpha_VaR = 0.995
alpha_ES = 0.99

In [3]:
#Sizes for training set, validation set, test set, and set size for Monte Carlo estimation of the risk measures
M_1 = 1500000
M_2 = 500000
M_3 = 500000
#ignore N or N_2 in the following. Was kept just in case, but not used.
N_2 = 1
M_MC = 500000
#size of the set of data points used to calculate an IS density
M_IS = 2000000

In [4]:
#Calculating simulated values of S_tau and simulated payoffs from standard normal random variable simulations
def data_gen(Z,V):
    #simulate S_tau under P
    S_tau = s_0 * np.exp( (mu-0.5*sigma**2)*tau + sigma*np.sqrt(tau)*Z)
    #Simulate S_T given S_tau under Q
    S_T = S_tau * np.exp( (r-0.5*sigma**2)*(T-tau) + sigma*np.sqrt(T-tau)*V)
    #Calculate corresponding simulated discounted payoffs
    P_T = np.exp(-r*(T-tau)) * np.maximum(K-S_T,0)
    return S_tau, P_T

#the function DT(Z,\theta)
def data_trans_IS(Z,IS):
    return Z*IS[1] + IS[0]

#The density function of Z
def f(y):
    return stats.norm.pdf(y, loc=0, scale=1)

#The density function of Z_\theta (note that \theta is replaced by x; this is needed for the least-squares solver to work), variance included in the parametrisation
def f_theta(x, y):
    return stats.norm.pdf(y, loc=x[0], scale=x[1])

#Put- und Call-Black-Scholes Formeln
def d1(K, t, x, sigma, r):
    return (np.log(x/K)+(r+0.5*sigma**2)*t)/(sigma*np.sqrt(t))

def d2(K, t, x, sigma, r):
    return (np.log(x/K)+(r-0.5*sigma**2)*t)/(sigma*np.sqrt(t))

def put_true(K, t, x, sigma, r): 
    return K*np.exp(-r*t)*stats.norm.cdf(-d2(K,t,x,sigma,r)) - x*stats.norm.cdf(-d1(K,t,x,sigma,r))

def call_true(K, t, x, sigma, r):
    return x*stats.norm.cdf(d1(K,t,x,sigma,r)) - K*np.exp(-r*t)*stats.norm.cdf(d2(K,t,x,sigma,r))

#true function l
def P_T_true(x):
    return put_true(K=K, t=T-tau, x=x, sigma=sigma, r=r)

#This function describes the approximation of the expression inside the sum of m_2(theta)
def g_q_alpha_hat_reweighted(x,L,q_alpha_hat):
    return np.sqrt(f(y=L[:,0])/f_theta(y=L[:,0],x=x))*(L[:,-1]>q_alpha_hat)

#bounds for the IS density parameters (for the mean parameter bounds are necessary, the standard deviation parameter however needs to be non-negative)
bnds_lower = [-np.inf, 0]
bnds_upper = [np.inf, np.inf]
bnds = (bnds_lower, bnds_upper)

In [5]:
#Generating realisations of standard normal random variables
Z_IS = np.random.normal(loc=0, scale=1, size=M_IS)
V_IS = np.random.normal(loc=0, scale=1, size=M_IS)

#Calculate the risk factor S_tau and the corresponding simulated payoffs P_T
S_tau_IS, P_T_IS = data_gen(Z=Z_IS, V=V_IS)

#Calculate realisations of L_hat from the training data set using the true function l
L_hat_IS = np.column_stack((Z_IS, P_T_true(S_tau_IS)))
L_hat_IS_sort = L_hat_IS[L_hat_IS[:,-1].argsort()[::-1]]

#Calculating the corresponding estimator for Value-at-Risk in order to approximate g
q_alpha_hat_IS = L_hat_IS_sort[int(M_IS*(1-alpha_IS)-1), -1]
print('q_alpha_hat_IS:',q_alpha_hat_IS)

#Calculating the (hopefully) approximately optimal \theta^* by minimising m_2 using the approximated g
IS_obj = optimize.least_squares(g_q_alpha_hat_reweighted, x0=np.array([0,1]), args=(L_hat_IS, q_alpha_hat_IS), bounds=bnds)
IS = IS_obj.x
print(IS)

q_alpha_hat_IS: 8.342059689048256


/scratch/slurm_tmpdir/job_20138784/ipykernel_3974210/2114570057.py:38: RuntimeWarning: divide by zero encountered in true_divide
  return np.sqrt(f(y=L[:,0])/f_theta(y=L[:,0],x=x))*(L[:,-1]>q_alpha_hat)
/scratch/slurm_tmpdir/job_20138784/ipykernel_3974210/2114570057.py:38: RuntimeWarning: overflow encountered in true_divide
  return np.sqrt(f(y=L[:,0])/f_theta(y=L[:,0],x=x))*(L[:,-1]>q_alpha_hat)
/scratch/slurm_tmpdir/job_20138784/ipykernel_3974210/2114570057.py:38: RuntimeWarning: invalid value encountered in multiply
  return np.sqrt(f(y=L[:,0])/f_theta(y=L[:,0],x=x))*(L[:,-1]>q_alpha_hat)


[-2.83649144  0.43991149]


In [6]:
IS_obj.message

'`ftol` termination condition is satisfied.'

Even though error messages appear the algorithm succesfully converged thus we take the result to be meaningful and proceed.

In [7]:
run = 1
for j in range(100):
    #Generating simulations for standard normal random variables for  Monte Carlo estimation of risk measures
    Z_MC = np.random.normal(loc=0, scale=1, size=M_MC)
    V_MC = np.random.normal(loc=0, scale=1, size=M_MC)
    #calculating DT(Z,\theta^*)
    Z_MC_IS = data_trans_IS(Z_MC,IS)

    #Calculate the risk factor S_tau and the corresponding simulated payoffs P_T with original and IS distribution
    S_tau_MC,P_T_MC = data_gen(Z=Z_MC,V=V_MC)
    S_tau_MC_IS,P_T_MC_IS = data_gen(Z=Z_MC_IS,V=V_MC)
    
    #computation of option price depending on the risk factor S_tau according to the models, i.e. computation of L_hat_i's with original distribution
    L_hat = P_T_true(x=S_tau_MC)
    L_hat_sort = np.sort(L_hat)[::-1]
    
    #calculation of Value-at-Risk and Expected Shortfall estimators without IS
    j_VaR = int(M_MC*(1-alpha_VaR)-1)
    VaR_hat = L_hat_sort[j_VaR]
    
    j_ES = int(M_MC*(1-alpha_ES)-1)
    ES_hat = (1/(1-alpha_ES)) * np.sum(L_hat_sort[0:j_ES-1])/M_MC + ( 1 - (j_ES-1)/((1-alpha_ES)*M_MC) )*L_hat_sort[j_ES]
    
    #computation of option price depending on the risk factor S_tau according to the models, i.e. computation of L_hat_i's with IS distribution
    L_hat_IS = P_T_true(x=S_tau_MC_IS)
    L_hat_c_IS = np.column_stack((Z_MC_IS, L_hat_IS))
    
    #calculation of Value-at-Risk and Expected Shortfall estimators with IS
    L_hat_c_sort_IS = L_hat_c_IS[L_hat_c_IS[:,-1].argsort()[::-1]]
    w = f(L_hat_c_sort_IS[:,0])/(M_MC*f_theta(x=IS, y=L_hat_c_sort_IS[:,0]))

    j_VaR = 0
    w_sum_tmp = 0
    while (w_sum_tmp <= (1-alpha_VaR) and j_VaR<M_MC):
        w_sum_tmp += w[j_VaR]
        j_VaR += 1
    VaR_hat_IS = L_hat_c_sort_IS[j_VaR,-1]

    j_ES = 0
    w_sum_tmp = 0
    while (w_sum_tmp <= (1-alpha_ES) and j_ES<M_MC):
        w_sum_tmp += w[j_ES]
        j_ES += 1
    ES_hat_IS = (1/(1-alpha_ES)) * np.sum(w[0:j_ES-1]*L_hat_c_sort_IS[0:j_ES-1,-1]) + ( 1 - (1 / (1-alpha_ES)) * np.sum(w[0:j_ES-1]) )*L_hat_c_sort_IS[j_ES,-1]
    
    #save results for further evaluation
    output = np.array([VaR_hat,VaR_hat_IS,ES_hat,ES_hat_IS])
    
    joblib.dump(output, filepath+'output'+str(j)+'_'+str(run)+'.joblib')
    #prints just for checking while the notebook is running
    print(j)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
